In [3]:
#imports
import pandas as pd
from rouge import Rouge
from tqdm.auto import tqdm
import numpy as np

In [4]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [6]:
df_new = df.iloc[:300]

In [13]:
df_new.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [9]:
model = "multi-qa-mpnet-base-dot-v1"

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
answer_llm = df_new.iloc[0].answer_llm

In [12]:
embedding_model.encode(answer_llm)[0]

-0.42244655

In [29]:
evaluation = []

for index, record in tqdm(df_new.iterrows()):
    answer_org = record['answer_orig']
    answer_llm = record['answer_llm']
    
    llm = embedding_model.encode(answer_org)
    orig = embedding_model.encode(answer_llm)
    
    dot_product = llm.dot(orig)
    evaluation.append(dot_product)

0it [00:00, ?it/s]

In [30]:
df_new["score"] = evaluation

/tmp/ipykernel_6101/2975825665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["score"] = evaluation


In [31]:
df_new["score"].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547923
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: score, dtype: float64

In [83]:
def cosine_similarity(df, embedding_model):
    similarity = []

    for index, record in tqdm(df.iterrows(), total=df.shape[0]):
        answer_org = record['answer_orig']
        answer_llm = record['answer_llm']
        
        # Encode the answers to get the vectors
        llm = embedding_model.encode(answer_llm)
        orig = embedding_model.encode(answer_org)
        
        # Compute norms for each vector
        norm_llm = np.sqrt(np.sum(llm ** 2))
        norm_orig = np.sqrt(np.sum(orig ** 2))
        
        # Normalize the vectors
        llm_norm = llm / norm_llm
        orig_norm = orig / norm_orig
        
        # Compute the cosine similarity
        dot_product = np.dot(llm_norm, orig_norm)
        similarity.append(dot_product)
    
    return similarity

In [81]:
df_new["cosine"] = cosine_similarity(df_new, embedding_model)

  0%|          | 0/300 [00:00<?, ?it/s]

/tmp/ipykernel_6101/3944168541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["cosine"] = cosine_similarity(df_new, embedding_model)


In [82]:
df_new["cosine"].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

In [62]:
# instantiate Rouge
rouge_scorer = Rouge()

In [66]:
r = df_new.iloc[10]

In [75]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [76]:
scores['rouge-1']['f']

0.45454544954545456

In [77]:
rouge_1_f1 = scores['rouge-1']['f']
rouge_2_f1 = scores['rouge-2']['f']
rouge_l_f1 = scores['rouge-l']['f']

In [79]:
average_f1 = (rouge_1_f1 + rouge_2_f1 + rouge_l_f1) / 3
average_f1

0.35490034990035496

In [86]:
rouge_2_f1_scores = []

for index, record in tqdm(df_new.iterrows(), total=df.shape[0]):
    scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    rouge_2_f1 = scores['rouge-2']['f']
    rouge_2_f1_scores.append(rouge_2_f1)

average_rouge_2_f1 = sum(rouge_2_f1_scores) / len(rouge_2_f1_scores)

  0%|          | 0/300 [00:00<?, ?it/s]

In [87]:
print(f"Average ROUGE-2 F1 Score across all records: {average_rouge_2_f1}")

Average ROUGE-2 F1 Score across all records: 0.20696501983423318
